This is used to clean up a merged datafram with nan's

In [32]:
import pandas as pd
import numpy as np

In [55]:
# Making a specific dataframe
d = {'Depth': [1, 2, 4 , 4, 5], 'Rock': [1, 3, 2, np.nan, 6], 'Roll': [3, 17, np.nan , 12, 4]}
df = pd.DataFrame(data=d).set_index('Depth')
df

,Rock,Roll
Depth,,
1,1.0,3.0
2,3.0,17.0
4,2.0,NaN
4,NaN,12.0
5,6.0,4.0


Let's merge the two rows with the same depth

In [44]:
# Checking to see if 
pd.concat(g for _, g in df.groupby("Depth") if len(g) > 1)

,Depth,Rock,Roll
2,4,2.0,NaN
3,4,NaN,12.0


This is from Jess Robertson on SWUNG. Thanks! https://github.com/jesserobertson

In [47]:
def merge_me(df):
    "Merge nan rows"
    records = []
    for depth, repeats in df.groupby('Depth'):
        if len(repeats) > 1:
            if any((~repeats.isna()).sum() > 1):
                # halt and catch fire
                raise ValueError(f'Need QA/QC at depth {depth}')
            else:
                record = dict(zip(repeats.columns, np.nansum(repeats, axis=0)))
                record['Depth'] = depth
                records.append(record)
        else:
            record = dict(zip(repeats.columns, repeats.iloc[0]))
            record['Depth'] = depth
            records.append(record)
    return pd.DataFrame.from_records(records)

In [58]:
# tests
df_ok = pd.DataFrame(
    [[0, 1, 2, 3], [1, 1, np.nan, np.nan], [1, np.nan, np.nan, 3], [2, 1, 2, 3]],
    columns=['Depth', 'a', 'b', 'c']
).set_index('Depth')
df_fails = pd.DataFrame(
    [[0, 1, 2, 3], [1, 1, np.nan, np.nan], [1, 2, 2, np.nan], [1, np.nan, np.nan, 3], [2, 1, 2, 3]],
    columns=['Depth', 'a', 'b', 'c']
).set_index('Depth')

merge_me(df_ok)     # works
merge_me(df_fails)  # fails with "ValueError: Need QA/QC at depth 1"

ValueError: Need QA/QC at depth 1

In [53]:
clean_df = merge_me(df)
clean_df

,a,b,c
Depth,,,
0,1.0,2.0,3.0
1,1.0,NaN,NaN
1,NaN,NaN,3.0
2,1.0,2.0,3.0
